In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings("ignore")

In [ ]:
data = pd.read_excel('/kaggle/input/telco-customer-churn-ibm-dataset/Telco_customer_churn.xlsx')

In [ ]:
df = data.copy()

In [ ]:
df.head(10)

In [ ]:
df['Total Charges'] = df['Total Charges'].replace(' ', np.nan )

In [ ]:
print(type(df['Total Charges']))

In [ ]:
df['Total Charges'] = df['Total Charges'].astype(float)

In [ ]:
print(type(df['Total Charges']))

In [ ]:
df['Total Charges'] = df['Total Charges'].fillna(2283.300441)

In [ ]:
df['Total Charges'].dtype

In [ ]:
df.isnull().sum()

In [ ]:
df[df['CustomerID'].duplicated()]

In [ ]:
df.info()

In [ ]:
dc = df.columns

In [ ]:
for i in dc:
    vc = df[i].value_counts()
    print(vc)
    print('*'*50)
    print(' '*50)

In [ ]:
df.head()

### Analysis Telco Customer Churn (IBM Dataset)

In [ ]:
sns.countplot(y=df['Churn Reason'])
plt.xticks(rotation=90)
plt.show()

Due to compitition and behavior of service provider is main reason for churning of Internat Services.

In [ ]:
cc = df.groupby('Churn Label')['City'].value_counts()

In [ ]:
most_churn_city = cc['Yes'].idxmax()
most_churn_count = cc['Yes'].max()

In [ ]:
print(f"Most churn city is {most_churn_city} with {most_churn_count} churn")

In [ ]:
cr = df[df['Churn Label'] == 'Yes']['City'].value_counts()
cr_2 = cr.head(20)

In [ ]:
sns.barplot(x=cr_2.values, y=cr_2.index, palette="viridis")
plt.show()

Internet services in Los Angeles are growing repidely. Internet Service providers in Los Angeles are very competing.

In [ ]:
sns.histplot(data=df, x="Tenure Months", hue="Churn Label", multiple="stack", palette="viridis")
plt.title("Distribution of Churn Label vs Tenure Months")
plt.show()

In [ ]:
sns.histplot(data=df, x="Contract", hue="Churn Label", multiple="stack", palette="YlGn")
plt.title("Distribution of Contracts and churn labels accordingly")
plt.show()

Monthly subscription is very easy to cancel. On the other hand, Two year subscription is very hard to cancel.

In [ ]:
sns.countplot(x=df['Internet Service'])
plt.title("Total Number of each Internet Service")
plt.show()

Most customers are using Fiber Optic Internet Services.

In [ ]:
df['Churn Label'].value_counts()

In [ ]:
sns.histplot(data=df, x="Internet Service", hue="Churn Label", multiple="stack", palette="Paired")
plt.title("Distribution of Internet Service Vs Churn Label")
plt.show()

And customers using Fiber optic generally cancel the subscription.

##### The lowest churn rate among customers without Internet services (but there are fewer such customers).

### Feature Engineering

In [ ]:
df.columns

In [ ]:
df = df.drop(['CustomerID', 'Count', 'Country', 'State', 'City', 'Zip Code', 'Lat Long', 'Latitude', 'Longitude', 'CLTV', 'Churn Value', 'Churn Score', 'Churn Reason'], axis=1)

df.head(4)

In [ ]:
df.shape

In [ ]:
y = df['Churn Label']

In [ ]:
y.replace(to_replace='Yes', value=1, inplace=True)
y.replace(to_replace='No', value=0, inplace=True)

In [ ]:
y

In [ ]:
df = df.drop('Churn Label', axis=1)

In [ ]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()

In [ ]:
df.columns

In [ ]:
en_c = df.drop(['Monthly Charges', 'Total Charges', 'Tenure Months'], axis=1)
col = en_c.columns
col

In [ ]:
for i in col:
    df[i] = le.fit_transform(df[i])

In [ ]:
df.head(4)

In [ ]:
xy = pd.concat([df, y], axis=1)
xy.corr()

In [ ]:
plt.figure(figsize=(18,16))

sns.heatmap(xy.corr(), annot=True)
plt.show()

In [ ]:
df.shape

In [ ]:
y.value_counts()

### Dataset is imbalance

Let us set the random state as 42.

In [ ]:
from imblearn.over_sampling import SMOTE

smt = SMOTE(random_state=42)

Xs, ys = smt.fit_resample(df,y)

In [ ]:
Xs.shape

In [ ]:
ys.shape

In [ ]:
ys.value_counts()

In [ ]:
from sklearn.model_selection import train_test_split, RandomizedSearchCV

X_train, X_test, y_train, y_test = train_test_split(Xs, ys, test_size=0.2, random_state=42)

### Model Selection

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier

In [ ]:
models = {
    'gb' : GradientBoostingClassifier(random_state=42),
    'xg' : XGBClassifier(random_state=42),
    'rf' : RandomForestClassifier(random_state=42),
    'lr' : LogisticRegression(random_state=42)
}

In [ ]:
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
from sklearn.metrics import roc_curve, roc_auc_score
from sklearn.feature_selection import RFE

In [ ]:
result = []
roc_model = {}

for name, model in models.items():
    model.fit(X_train, y_train)

    yp = model.predict(X_test)

    acs = accuracy_score(yp, y_test)
    f1 = f1_score(yp, y_test)
    ps = precision_score(yp,y_test)
    rs = recall_score(yp, y_test)
    fpr, tpr, thresholds = roc_curve(yp, y_test)
    auc_score = roc_auc_score(yp, y_test)

    roc_model[name] = (fpr, tpr, thresholds)

    result.append({
        'Model' : name,
        'Accuracy Score' : acs,
        'Recall' : rs,
        'f1' : f1,
        'Precision Score' : ps
    })

In [ ]:
print(roc_model)

In [ ]:
auc_score

In [ ]:
res = pd.DataFrame(result)

In [ ]:
res

In [ ]:
roc_model['xg'][0]

In [ ]:
# For Random Forest
plt.figure(figsize=(8, 6))
plt.plot(roc_model['rf'][0], roc_model['rf'][1], color='blue', label=f'AUC = {auc_score:.2f}')
plt.plot([0, 1], [0, 1], color='red', linestyle='--') # Diagonal Line
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve')
plt.legend(loc='lower right')
plt.grid()
plt.show()

This is the ROC curve before Hypertuning.

### HyperParameter Tuning

In [ ]:
prm = {
    'n_estimators' : [50, 100, 150],
    'criterion' : ['gini', 'entropy', 'log_loss'],
    'max_depth' : [2, 3, 4, 5],
    'max_features' : ['sqrt', 'log2', None]
}

In [ ]:
rf = RandomForestClassifier(random_state=42)

In [ ]:
rs = RandomizedSearchCV(
    estimator= rf,
    param_distributions= prm,
    n_iter=100,
    n_jobs=-1,
    random_state=42,
    cv=5,
    scoring='f1'
)

In [ ]:
rs.fit(X_train, y_train)

In [ ]:
rs.best_score_

In [ ]:
rs.best_params_

### Confusion matrix and ROC curve

In [ ]:
ryp = rs.predict(X_test)

In [ ]:
fpr, tpr, thresholds = roc_curve(ryp, y_test)

In [ ]:
auc_score = roc_auc_score(ryp, y_test)
print(f"AUC Score: {auc_score:.2f}")

After HyperTuning Parameters

In [ ]:
plt.figure(figsize=(8, 6))
plt.plot(fpr, tpr, color='blue', label=f'AUC = {auc_score:.2f}')
plt.plot([0, 1], [0, 1], color='red', linestyle='--')  # Diagonal line
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve')
plt.legend(loc='lower right')
plt.grid()
plt.show()

In [ ]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

In [ ]:
cm = confusion_matrix(y_test, [1 if p > 0.5 else 0 for p in ryp])

In [ ]:
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=['No Churn', 'Churn'])
disp.plot(cmap="Blues")
plt.title("Confusion Matrix")
plt.show()

Actual vs. Predicted class distribution.

### Feature Importance

In [ ]:
rf = RandomForestClassifier(random_state=42)

In [ ]:
rf.fit(X_train, y_train)

In [ ]:
rf_yp = rf.predict(X_test)

In [ ]:
feature_importances = rf.feature_importances_
feature_importances

In [ ]:
feature_importances.shape

In [ ]:
feature_names = X_test.columns
feature_names

In [ ]:
feature_names.shape

In [ ]:
importance_df = pd.DataFrame({'Feature': feature_names, 'Importance': feature_importances})
importance_df = importance_df.sort_values(by='Importance', ascending=False)

In [ ]:
plt.figure(figsize=(10, 6))
plt.barh(importance_df['Feature'], importance_df['Importance'], color='skyblue')
plt.gca().invert_yaxis()
plt.xlabel('Importance')
plt.ylabel('Feature')
plt.title('Feature Importance')
plt.show()

Monthly Charges and Total Charges drive the model's predictions the most.